# 音訊轉文字工具 (上傳檔案版)

這個 notebook 可以：
- 上傳音訊/影片檔案
- 使用 Whisper 模型將音訊轉換成中文文字
- 支援多種檔案格式
- 輸出包含時間戳記的文字檔

## 修改使用 GPU 運算
執行階段 -> 變更執行類型 -> 選T4 GPU

## 使用方法
1. 依序執行每個 cell
2. 上傳你的音訊/影片檔案
3. 等待處理完成後下載結果

## 1. 安裝必要套件

In [ ]:
# 安裝所需套件
!pip install openai-whisper
!pip install torch torchvision torchaudio
# 確保 FFmpeg 可用 (Colab 通常已經安裝)
!apt update && apt install -y ffmpeg

## 2. 匯入套件和初始設定

In [ ]:
import whisper
import glob
import os
from google.colab import files
import shutil

# 建立必要資料夾
os.makedirs("./subtitles/", exist_ok=True)
os.makedirs("./uploads/", exist_ok=True)

print("資料夾建立完成！")
print("支援的檔案格式：")
print("音訊：mp3, wav, flac, m4a, ogg, wma, aac")
print("影片：mp4, avi, mov, mkv, wmv, flv, webm")

## 3. 載入 Whisper 模型

In [ ]:
# 載入 Whisper 模型 (第一次會需要下載模型檔案)
print("正在載入 Whisper 模型...")
model = whisper.load_model("medium")
print("模型載入完成！")

# 設定路徑
export_dir = "./subtitles/"
upload_dir = "./uploads/"

## 4. 定義核心函數

In [ ]:
def create_txt(fn, verbose: bool = False):
    """將音訊檔案轉換成文字檔"""
    print(f"正在轉換: {os.path.basename(fn)}")
    
    try:
        result = model.transcribe(fn, language="zh", verbose=verbose)
        
        # 修正路徑分隔符號問題 (適用於不同作業系統)
        filename = os.path.basename(fn)
        # 移除所有可能的副檔名
        base_name = filename.split('.')[0]
        export_fn = os.path.join(export_dir, base_name + '.txt')
        
        def format_timestamp(seconds):
            seconds = int(seconds)
            minutes, seconds = divmod(seconds, 60)
            hours, minutes = divmod(minutes, 60)
            return f"{hours:02}:{minutes:02}:{seconds:02}"
        
        with open(export_fn, "w", encoding="utf-8") as file:
            for segment in result["segments"]:
                start = format_timestamp(segment["start"])
                end = format_timestamp(segment["end"])
                text = segment["text"]
                file.write(f"[{start} --> {end}] {text}\n")
        
        print(f"✅ 轉換完成: {os.path.basename(export_fn)}")
        return export_fn
        
    except Exception as e:
        print(f"❌ 轉換失敗: {os.path.basename(fn)}, 錯誤: {e}")
        return None

def get_supported_files(directory):
    """取得指定目錄中所有支援的音訊/影片檔案"""
    supported_extensions = [
        # 音訊格式
        '*.mp3', '*.wav', '*.flac', '*.m4a', '*.ogg', '*.wma', '*.aac',
        # 影片格式
        '*.mp4', '*.avi', '*.mov', '*.mkv', '*.wmv', '*.flv', '*.webm'
    ]
    
    files = []
    for ext in supported_extensions:
        files.extend(glob.glob(os.path.join(directory, ext)))
        files.extend(glob.glob(os.path.join(directory, ext.upper())))
    
    return files

print("函數定義完成！")

## 5. 上傳檔案

In [ ]:
# 清空上傳資料夾
if os.path.exists(upload_dir):
    shutil.rmtree(upload_dir)
os.makedirs(upload_dir, exist_ok=True)

print("請選擇要上傳的音訊/影片檔案：")
print("支援格式：mp3, wav, flac, m4a, ogg, wma, aac, mp4, avi, mov, mkv, wmv, flv, webm")
print("\n⚠️  注意：可以一次選擇多個檔案")

uploaded = files.upload()

# 將上傳的檔案移動到指定資料夾
for filename in uploaded.keys():
    src = filename
    dst = os.path.join(upload_dir, filename)
    shutil.move(src, dst)
    print(f"✅ 已上傳: {filename}")

print(f"\n📁 總共上傳了 {len(uploaded)} 個檔案")

## 6. 處理上傳的檔案

In [ ]:
# 取得所有支援的檔案
uploaded_files = get_supported_files(upload_dir)

if not uploaded_files:
    print("❌ 沒有找到支援的檔案格式")
    print("請確認上傳的檔案是以下格式之一：")
    print("音訊：mp3, wav, flac, m4a, ogg, wma, aac")
    print("影片：mp4, avi, mov, mkv, wmv, flv, webm")
else:
    print(f"找到 {len(uploaded_files)} 個支援的檔案：")
    for file in uploaded_files:
        print(f"  - {os.path.basename(file)}")
    
    # 開始轉換
    print("\n開始轉換音訊為文字...")
    successful_conversions = []
    
    for i, file_path in enumerate(uploaded_files, 1):
        print(f"\n處理第 {i}/{len(uploaded_files)} 個檔案...")
        result = create_txt(file_path)
        if result:
            successful_conversions.append(result)
    
    print(f"\n🎉 處理完成！")
    print(f"成功轉換 {len(successful_conversions)}/{len(uploaded_files)} 個檔案")

## 7. 查看結果

In [ ]:
# 查看上傳的檔案
print("📁 上傳的檔案:")
uploaded_files = get_supported_files(upload_dir)
if uploaded_files:
    for file in uploaded_files:
        file_size = os.path.getsize(file) / (1024 * 1024)  # MB
        print(f"  - {os.path.basename(file)} ({file_size:.2f} MB)")
else:
    print("  (沒有檔案)")

print("\n📄 生成的文字檔案:")
txt_files = glob.glob("./subtitles/*.txt")
if txt_files:
    for file in txt_files:
        print(f"  - {os.path.basename(file)}")
else:
    print("  (沒有檔案)")

print(f"\n📊 統計:")
print(f"  - 上傳檔案: {len(uploaded_files)} 個")
print(f"  - 文字檔案: {len(txt_files)} 個")

## 8. 預覽文字內容

In [ ]:
# 預覽生成的文字檔案內容
txt_files = glob.glob("./subtitles/*.txt")

if txt_files:
    for file in txt_files:
        print(f"\n{'='*50}")
        print(f"檔案: {os.path.basename(file)}")
        print(f"{'='*50}")
        
        with open(file, 'r', encoding='utf-8') as f:
            content = f.read()
            # 只顯示前 500 個字元
            if len(content) > 500:
                print(content[:500] + "\n\n... (內容過長，僅顯示前 500 字元)")
            else:
                print(content)
else:
    print("沒有找到文字檔案，請先執行步驟 5 和 6")

## 9. 下載結果檔案

In [ ]:
# 下載文字檔案到本地
import zipfile

txt_files = glob.glob("./subtitles/*.txt")

if txt_files:
    if len(txt_files) == 1:
        # 如果只有一個檔案，直接下載
        print(f"下載檔案: {os.path.basename(txt_files[0])}")
        files.download(txt_files[0])
    else:
        # 如果有多個檔案，打包成 zip 檔案
        zip_filename = "transcriptions.zip"
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in txt_files:
                zipf.write(file, os.path.basename(file))
        
        print(f"已打包 {len(txt_files)} 個檔案到 {zip_filename}")
        files.download(zip_filename)
else:
    print("沒有找到文字檔案可供下載")

## 10. 批次上傳 (選用功能)

In [ ]:
# 如果你有很多檔案需要處理，可以先打包成 zip 檔上傳
# 然後使用這個 cell 來解壓縮

print("上傳 zip 檔案 (包含多個音訊/影片檔案):")
uploaded_zip = files.upload()

# 解壓縮
for zip_filename in uploaded_zip.keys():
    if zip_filename.endswith('.zip'):
        print(f"正在解壓縮: {zip_filename}")
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(upload_dir)
        print(f"解壓縮完成！")
        
        # 顯示解壓縮的檔案
        extracted_files = get_supported_files(upload_dir)
        print(f"找到 {len(extracted_files)} 個支援的檔案：")
        for file in extracted_files:
            print(f"  - {os.path.basename(file)}")
    else:
        print(f"⚠️  {zip_filename} 不是 zip 檔案，跳過")

## 📝 使用說明

### 支援的檔案格式
- **音訊格式**: MP3, WAV, FLAC, M4A, OGG, WMA, AAC
- **影片格式**: MP4, AVI, MOV, MKV, WMV, FLV, WebM

### 使用流程
1. 執行步驟 1-4 進行初始化
2. 執行步驟 5 上傳檔案
3. 執行步驟 6 處理檔案
4. 執行步驟 7-9 查看和下載結果

### 進階功能
- **批次處理**: 可以一次上傳多個檔案
- **ZIP 上傳**: 如果檔案很多，可以打包成 zip 檔上傳 (步驟 10)
- **模型選擇**: 可以在步驟 3 中修改模型大小

### 模型選擇建議
- `"tiny"`: 最快速，但準確度最低
- `"base"`: 速度快，準確度一般
- `"small"`: 平衡速度和準確度
- `"medium"`: 推薦選擇，準確度較高
- `"large"`: 最高準確度，但速度最慢

### 注意事項
- 檔案大小限制：Google Colab 有上傳大小限制
- 處理時間：取決於檔案長度和模型大小
- 語言設定：目前設定為中文，可以修改或移除讓模型自動偵測
- GPU 加速：建議使用 GPU 運算環境提升速度

### 常見問題
- **檔案格式不支援**: 確認檔案副檔名是否正確
- **轉換失敗**: 可能是檔案損壞或格式特殊，嘗試用其他軟體轉換格式
- **記憶體不足**: 嘗試使用較小的模型或分批處理